In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor

## Load & Cleaning Dataset

In [16]:
## Load data into dataframes
df_2020_titer = pd.read_csv("https://www.cmi-pb.org/downloads/cmipb_challenge_datasets/current/2nd_challenge/raw_datasets/training_data/2020LD_plasma_ab_titer.tsv", sep='\t')
df_2020_specimen = pd.read_csv("https://www.cmi-pb.org/downloads/cmipb_challenge_datasets/current/2nd_challenge/raw_datasets/training_data/2020LD_specimen.tsv", sep='\t')
df_2020_subject = pd.read_csv("https://www.cmi-pb.org/downloads/cmipb_challenge_datasets/current/2nd_challenge/raw_datasets/training_data/2020LD_subject.tsv", sep='\t')
df_2021_titer = pd.read_csv("https://www.cmi-pb.org/downloads/cmipb_challenge_datasets/current/2nd_challenge/raw_datasets/training_data/2021LD_plasma_ab_titer.tsv", sep='\t')
df_2021_specimen = pd.read_csv("https://www.cmi-pb.org/downloads/cmipb_challenge_datasets/current/2nd_challenge/raw_datasets/training_data/2021LD_specimen.tsv", sep='\t')
df_2021_subject = pd.read_csv("https://www.cmi-pb.org/downloads/cmipb_challenge_datasets/current/2nd_challenge/raw_datasets/training_data/2021LD_subject.tsv", sep='\t')
df_2022_titer = pd.read_csv("https://www.cmi-pb.org/downloads/cmipb_challenge_datasets/current/2nd_challenge/raw_datasets/prediction_data/2022BD_plasma_ab_titer.tsv", sep='\t')
df_2022_specimen = pd.read_csv("https://www.cmi-pb.org/downloads/cmipb_challenge_datasets/current/2nd_challenge/raw_datasets/prediction_data/2022BD_specimen.tsv", sep='\t')
df_2022_subject = pd.read_csv("https://www.cmi-pb.org/downloads/cmipb_challenge_datasets/current/2nd_challenge/raw_datasets/prediction_data/2022BD_subject.tsv", sep='\t')

In [34]:
## cleaning the titer data to only include IgG isotypes and PT antigens
def clean_df_titer(df):
    df.dropna(inplace=True)
    
    # extract row with IgG in the "isotype" column and PT in the "antigen" column
    df = df[(df['isotype'] == 'IgG') & (df['antigen'] == 'PT')]
    df = df[['specimen_id', 'MFI_normalised']].rename(columns={'MFI_normalised': 'IgG_PT'})
    return df

In [83]:
## cleaning the subject/specimen dataset to get an age column
def clean_df_subject(df):
    
    ## Get age column
    df['year_of_birth'] = pd.to_numeric(df['year_of_birth'].str[:4])
    df['date_of_boost'] = pd.to_numeric(df['date_of_boost'].str[:4])
    df['age'] = df['date_of_boost'] - df['year_of_birth']
    return df

In [84]:
## concatenating 2020 and 2021 titer datasets
titers = pd.concat([clean_df_titer(df_2020_titer),clean_df_titer(df_2021_titer)])

In [52]:
## concatenating 2020 and 2021 subject & specimen datasets
subject = clean_df_subject(pd.concat([pd.merge(df_2020_specimen,df_2020_subject, on= 'subject_id'),
pd.merge(df_2021_specimen,df_2021_subject, on= 'subject_id')],ignore_index=True))

In [53]:
## merging the titer & subject/specimen datasets to one dataframe
IgG = subject.merge(titers, on='specimen_id')

In [55]:
## getting rid of unnecessary columns
IgG = IgG[['subject_id',
           'specimen_id',
           'infancy_vac',
           'biological_sex',
           'age',
           'year_of_birth',
           'date_of_boost',
           'actual_day_relative_to_boost',
           'planned_day_relative_to_boost',
           'ethnicity',
           'race',
           'dataset',
           'specimen_type',
           'visit',
           'IgG_PT']]

In [56]:
IgG.head()

,subject_id,specimen_id,infancy_vac,biological_sex,age,year_of_birth,date_of_boost,actual_day_relative_to_boost,planned_day_relative_to_boost,ethnicity,race,dataset,specimen_type,visit,IgG_PT
0,1,1,wP,Female,30,1986,2016,-3,0,Not Hispanic or Latino,White,2020_dataset,Blood,1,3.736992
1,1,3,wP,Female,30,1986,2016,3,3,Not Hispanic or Latino,White,2020_dataset,Blood,3,2.255534
2,1,4,wP,Female,30,1986,2016,7,7,Not Hispanic or Latino,White,2020_dataset,Blood,4,3.250369
3,1,5,wP,Female,30,1986,2016,11,14,Not Hispanic or Latino,White,2020_dataset,Blood,5,10.874112
4,1,6,wP,Female,30,1986,2016,32,30,Not Hispanic or Latino,White,2020_dataset,Blood,6,12.513860


## Data Feature Selection & Cleaning

In [85]:
## getting separate rows for day 0 MFI_normalized results and day 14 MFI_normalized results for IgG_PT 
IgG_d14 = IgG[IgG['planned_day_relative_to_boost'] == 14.0]
IgG_d0 = IgG[IgG['planned_day_relative_to_boost'] == 0.0][['subject_id', 'IgG_PT']]
IgG_d0 = IgG_d0.rename(columns={'subject_id': 'subject_id', 'IgG_PT': 'IgG_PT_d0'})
IgG_d14 = IgG_d14.merge(IgG_d0, on='subject_id')

In [86]:
IgG_d14 = IgG_d14[['infancy_vac', 'biological_sex', 'year_of_birth', 'ethnicity', 'race', 'visit', 'IgG_PT_d0', 'IgG_PT']]

In [87]:
IgG_d14.head()

,infancy_vac,biological_sex,year_of_birth,ethnicity,race,visit,IgG_PT_d0,IgG_PT
0,wP,Female,1986,Not Hispanic or Latino,White,5,3.736992,10.874112
1,wP,Female,1983,Unknown,White,5,1.096366,7.041547
2,wP,Male,1988,Not Hispanic or Latino,Asian,5,2.046671,7.896541
3,wP,Male,1991,Not Hispanic or Latino,Asian,5,3.798007,5.327203
4,wP,Female,1988,Not Hispanic or Latino,White,5,0.213328,9.128886


One Hot Encoding and mapping string data into numerical type for Regression training

In [61]:
IgG_d14['infancy_vac'] = IgG_d14['infancy_vac'].map({'wP':0, 'aP':1})

In [62]:
IgG_d14['biological_sex'] = IgG_d14['biological_sex'].map({'Female':0, 'Male':1})

In [63]:
IgG_d14['ethnicity'] = IgG_d14['ethnicity'].map({'Not Hispanic or Latino':0, 
                                                   'Hispanic or Latino':1,
                                                   'Unknown':2})

In [64]:
IgG_d14['race'] = IgG_d14['race'].map({'White':0, 
                                       'Asian':1,
                                       'Unknown or Not Reported': 2,
                                       'More Than One Race': 2,
                                       'Black or African American': 3,
                                       'Native Hawaiian or Other Pacific Islander': 4,
                                       'American Indian/Alaska Native':5
                                      })
IgG_d14.head()

,infancy_vac,biological_sex,year_of_birth,ethnicity,race,visit,IgG_PT_d0,IgG_PT
0,0,0,1986,0,0,5,3.736992,10.874112
1,0,0,1983,2,0,5,1.096366,7.041547
2,0,1,1988,0,1,5,2.046671,7.896541
3,0,1,1991,0,1,5,3.798007,5.327203
4,0,0,1988,0,0,5,0.213328,9.128886


## Training and Evaluating Models

In [65]:
X = IgG_d14.drop('IgG_PT', axis = 1)
y = IgG_d14['IgG_PT']

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

First we will test a simple naive model, Linear Regression, to get a baseline of how a model should at least perform

In [68]:
reg = LinearRegression().fit(X, y)

In [69]:
reg.score(X, y)

0.15758400689208552

The MSE, MAE, and R2 scores were mediocre at best, and could defintely be improved upon

In [70]:
Y_pred = reg.predict(X)

In [71]:
mse = mean_squared_error(y, Y_pred)
mae = mean_absolute_error(y, Y_pred)
r2 = r2_score(y, Y_pred)
mse, mae, r2

(21.784966938075527, 3.6549025999603764, 0.15758400689208552)

Next, we will test ElasticNet, a combination of Ridge and Lasso Regression

In [72]:
model = ElasticNet()
model.fit(X, y)

ElasticNet()

In [73]:
Y_pred = model.predict(X)

In [74]:
mse = mean_squared_error(y, Y_pred)
mae = mean_absolute_error(y, Y_pred)
r2 = r2_score(y, Y_pred)
mse, mae, r2

(22.258354631653216, 3.71491243531239, 0.1392782933629403)

Since our training data we are working with is fairly constricted in size, we will take a look at how KNeighborsRegressor will perform

In [75]:
kn = KNeighborsRegressor()
kn.fit(X, y)

KNeighborsRegressor()

In [76]:
Y_pred = kn.predict(X)

In [77]:
mse = mean_squared_error(y, Y_pred)
mae = mean_absolute_error(y, Y_pred)
r2 = r2_score(y, Y_pred)
mse, mae, r2

(14.728803426252703, 2.984719020985153, 0.4304430389595064)

The results are promising with a R2 score of 0.43 on a scale from 0 to 1, alot better than our baseline with Linear Regression of 0.15. The MSE score was also improved upon.

Lastly, we will test DecisionTreeRegressor to see how it performs in comparison to KNeighborsRegressor

In [78]:
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_3 = DecisionTreeRegressor(max_depth=10)
regr_1.fit(X, y)
regr_2.fit(X, y)
regr_3.fit(X, y)

DecisionTreeRegressor(max_depth=10)

In [79]:
y_1 = regr_1.predict(X)
y_2 = regr_2.predict(X)
y_3 = regr_3.predict(X)

In [80]:
mse = mean_squared_error(y, y_1)
mae = mean_absolute_error(y, y_1)
r2 = r2_score(y, y_1)
mse, mae, r2

(15.927582684728469, 3.0081246225292047, 0.3840867225869996)

In [81]:
mse = mean_squared_error(y, y_2)
mae = mean_absolute_error(y, y_2)
r2 = r2_score(y, y_2)
mse, mae, r2

(11.876035290340502, 2.2756843781814236, 0.5407584463297501)

In [82]:
mse = mean_squared_error(y, y_3)
mae = mean_absolute_error(y, y_3)
r2 = r2_score(y, y_3)
mse, mae, r2

(0.5575911007495251, 0.33231507317888076, 0.9784381742592839)

## 2022 Validation Predictions

#### Loading in 2022 prediction datasets: titer, specimen, and subject

In [128]:
titers_pred = clean_df_titer(df_2022_titer)

In [129]:
subject_pred = clean_df_subject(pd.merge(df_2022_specimen,df_2022_subject, on= 'subject_id'))

In [130]:
IgG = subject_pred.merge(titers_pred, on='specimen_id', how = 'outer')

In [131]:
IgG.columns

Index(['specimen_id', 'subject_id', 'actual_day_relative_to_boost',
       'planned_day_relative_to_boost', 'specimen_type', 'visit',
       'infancy_vac', 'biological_sex', 'ethnicity', 'race', 'year_of_birth',
       'date_of_boost', 'dataset', 'age', 'IgG_PT'],
      dtype='object')

In [132]:
IgG = IgG[['subject_id',
           'specimen_id',
           'infancy_vac',
           'biological_sex',
           'age',
           'year_of_birth',
           'date_of_boost',
           'actual_day_relative_to_boost',
           'planned_day_relative_to_boost',
           'ethnicity',
           'race',
           'dataset',
           'specimen_type',
           'visit',
           'IgG_PT']]

In [133]:
IgG = IgG[IgG['planned_day_relative_to_boost'].isin([0])]

In [134]:
IgG_d= IgG[['infancy_vac', 'biological_sex', 'year_of_birth', 'ethnicity', 'race', 'visit', 'IgG_PT']].rename(columns={'IgG_PT': 'IgG_PT_d0'})

#### Data Feature Selection & Cleaning

In [135]:
IgG_d['infancy_vac'] = IgG_d['infancy_vac'].map({'wP':0, 'aP':1})

In [136]:
IgG_d['biological_sex'] = IgG_d['biological_sex'].map({'Female':0, 'Male':1})

In [137]:
IgG_d['ethnicity'] = IgG_d['ethnicity'].map({'Not Hispanic or Latino':0, 
                                                   'Hispanic or Latino':1,
                                                   'Unknown':2})

In [138]:
IgG_d['race'] = IgG_d['race'].map({'White':0, 
                                       'Asian':1,
                                       'Unknown or Not Reported': 2,
                                       'More Than One Race': 2,
                                       'Black or African American': 3,
                                       'Native Hawaiian or Other Pacific Islander': 4,
                                       'American Indian/Alaska Native':5
                                      })
IgG_d.head()

,infancy_vac,biological_sex,year_of_birth,ethnicity,race,visit,IgG_PT_d0
2,0,1,1986,0,0,3,1.060618
12,0,0,1993,0,0,5,1.309938
22,1,0,1999,1,2,5,1.196227
32,1,0,2001,0,0,5,0.967752
42,1,1,2003,0,0,3,1.651583


#### Getting predictions from our Model

In [143]:
y_pred = regr_3.predict(IgG_d)

In [146]:
rank = [abs(sorted(y_3).index(x)-19) for x in y_3]

In [147]:
IgG['1.1) IgG-PT-D14-titer-Rank'] = rank

In [148]:
IgG[['subject_id', 'age', 'biological_sex', 'infancy_vac', '1.1) IgG-PT-D14-titer-Rank']]

,subject_id,age,biological_sex,infancy_vac,1.1) IgG-PT-D14-titer-Rank
2,97,35,Male,wP,6
12,98,28,Female,wP,4
22,99,22,Female,aP,8
32,100,20,Female,aP,17
42,101,18,Male,aP,3
52,102,18,Male,aP,3
62,103,27,Female,wP,1
72,104,32,Female,wP,12
81,105,27,Female,wP,11
91,106,25,Female,aP,14
